In [1]:
%run Requirements.ipynb

24/06/04 16:49:36 WARN Utils: Your hostname, Ronits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
24/06/04 16:49:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/04 16:49:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.sql(
    """SELECT DISTINCT * FROM PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/AppleMusic`"""
)
artist_names = df.select("Artist Name").distinct().rdd.flatMap(lambda x: x).collect()

api_key = News_API.get('API_KEY')

def fetch_news_for_artist(artist_name):
    url = f"https://newsapi.org/v2/everything?q={artist_name}&pageSize=10&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get("articles", [])
    else:
        return []

all_articles = []
for artist in artist_names:
    articles = fetch_news_for_artist(artist)
    for article in articles:
        article['Artist Name'] = artist
    all_articles.extend(articles)

df_news = spark.createDataFrame(all_articles)
df_news = df_news.select("Artist Name", "author", "description", "title", "url")
df_news.show()
# df_news.write.mode('overwrite').format('parquet').save('Mart/AppleMusic-Details')
# df_news.write.mode('overwrite').format('parquet').saveAsTable('AppleMusicDetails')

24/06/04 16:49:40 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/06/04 16:49:40 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/06/04 16:49:42 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/06/04 16:49:42 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore ronitguptaaa@127.0.0.1
24/06/04 16:49:42 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
24/06/04 16:49:43 WARN ObjectStore: Failed to get database parquet, returning NoSuchObjectException


PySparkValueError: [CANNOT_INFER_EMPTY_SCHEMA] Can not infer schema from empty dataset.